In [1]:
# ! pip install deepctr==0.8.7 --no-deps
# ! pip install torch==1.7.0 torchvision==0.8.1 
# ! pip install tensorflow-gpu==1.13.1
# ! pip install numba

In [2]:
import sys
sys.path.append('..')
sys.path.append('../../config/')
from config_prosper import *
import os
import gc
import pandas as pd
import numpy as np
import tensorflow as tf

from time import time
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names,VarLenSparseFeat,build_input_features,input_from_feature_columns

from mytools.utils.myfile import savePkl,loadPkl
from mmoe_tf import MMOE,MMOE_FefM,MMOE_mutihead,Shared_Bottom,PLE_CGC_FEFM,MMOE_FefM_multihead
from evaluation import evaluate_deepctr
from tensorflow.python.keras.utils import multi_gpu_model
from tqdm import tqdm as tqdm
import warnings
import tensorflow as tf
print(tf.test.is_gpu_available())

BASE_DIR(目录): /home/tione/notebook
True


In [3]:
# GPU相关设置
warnings.filterwarnings('ignore')
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# 设置GPU按需增长
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
SEED = 200

In [4]:
def loadFeedinfo():
    feed = loadPkl(FEED_INFO_DEAL)
    feed[["bgm_song_id", "bgm_singer_id"]] += 1  # 0 用于填未知
    feed[["bgm_song_id", "bgm_singer_id", "videoplayseconds"]] = \
        feed[["bgm_song_id", "bgm_singer_id", "videoplayseconds"]].fillna(0)
    feed['bgm_song_id'] = feed['bgm_song_id'].astype('int64')
    feed['bgm_singer_id'] = feed['bgm_singer_id'].astype('int64')
    print('feedinfo loading over...')
    return feed
def myLeftjoin(left,right,on):
    return left.merge(right[right[on].isin(left[on])].set_index(on),how='left',left_on=on,right_index=True)


In [5]:
class myDataGenerator_base(tf.keras.utils.Sequence):
    def __init__(self, data: pd.DataFrame,batch_size=2048, shuffle=True,mode = 'train'):
        
        
        assert mode == 'train' or mode == 'test'
        if mode == 'test' and shuffle == True :
            raise ValueError('测试数据打乱了！')
            
        self.data = data.copy()
        self.data = self.data.reset_index(drop = True)
        self.target = ACTION_LIST
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(self.data.shape[0])
        self.feedinfo = loadFeedinfo()
        self.sparse_features = list(set(['userid', 'feedid', 'authorid', 'bgm_song_id', 'bgm_singer_id' 
                  ] +  [x for x in self.feedinfo.columns if 'manual_tag_list' in x 
                  ] + [x for x in self.feedinfo.columns if 'manual_keyword_list' in x 
                  ] + [x for x in self.feedinfo.columns if 'machine_keyword_list' in x]))
        
        self.var_len_features = ['manual_tag_list', 'manual_keyword_list', 'machine_keyword_list'] 
        self.dense_features = ['videoplayseconds',]
        
        
        
        if mode == 'train':
            self.dnn_feature_columns = self.getFeatureColumns()
            self.feature_names = get_feature_names(self.dnn_feature_columns)
            self.feature_index = build_input_features(self.dnn_feature_columns)
            savePkl(self.dnn_feature_columns,os.path.join(MODEL_PATH,'feature_columns_base.pkl'))
            print('feature columns have saved')
        else :
            self.dnn_feature_columns = loadPkl(os.path.join(MODEL_PATH,'feature_columns_base.pkl'))
            self.feature_names = get_feature_names(self.dnn_feature_columns)
            self.feature_index = build_input_features(self.dnn_feature_columns)
            print('load feature columns' ,os.path.join(MODEL_PATH,'feature_columns_base.pkl'))
        
        if self.shuffle:
            print('shuffle data index ing...')
            np.random.shuffle(self.indexes)

    def __len__(self):

        return (self.data.shape[0] // self.batch_size) + 1

    def __getitem__(self, index):
        batch_indexs = self.indexes[index * self.batch_size:(index + 1) *
                                    self.batch_size]
        batch_data = self.data.iloc[batch_indexs, :]
        
        return self.get_feature_on_batch(batch_data)

    def on_epoch_end(self):
        if self.shuffle:
            print('shuffle data index ing...')
            np.random.shuffle(self.indexes)
    def on_epoch_begain(self):
        if self.shuffle:
            print('shuffle data index ing...')
            np.random.shuffle(self.indexes)

    def get_feature_on_batch(self, batch):
        
        batch = batch.merge(self.feedinfo[[ x for x in self.feedinfo.columns if x in self.var_len_features + self.sparse_features + self.dense_features]],
                            how='left',
                            on='feedid')             
        x = {name: batch[name].values for name in self.feature_names}
        for col in ['manual_tag_list','manual_keyword_list','machine_keyword_list']:
            x[col] = np.array(batch[col].tolist())
        y = [batch[y].values for y in ACTION_LIST]
        return x,y
        
    def getFeatureColumns(self,):
        embedding_dim = 16
        sparse_features = [ x for x in self.sparse_features if '_list' not in x] #排除变长特征的单独列
        dense_features = self.dense_features 
         
        ### user id  and varlen
        userid_columns = [
            SparseFeat('userid',
                       vocabulary_size=USERID_MAX,
                       embedding_dim=embedding_dim)
        ]
        
        tag_columns = [
            VarLenSparseFeat(SparseFeat('manual_tag_list',
                                        vocabulary_size=TAG_MAX,
                                        embedding_dim=embedding_dim),
                             maxlen=4)
        ]
        
        key_words_columns = [
            VarLenSparseFeat(SparseFeat('manual_keyword_list',
                                        vocabulary_size=KEY_WORDS_MAX,
                                        embedding_dim=embedding_dim),
                             maxlen=4),
            VarLenSparseFeat(SparseFeat('machine_keyword_list',
                                        vocabulary_size=KEY_WORDS_MAX,
                                        embedding_dim=embedding_dim),
                             maxlen=4),
        ]
        
        # sparse
        fixlen_feature_columns = [
            SparseFeat(feat,
                       vocabulary_size=self.feedinfo[feat].max() + 1,
                       embedding_dim=embedding_dim) for feat in sparse_features
            if feat !='userid'
        ] + [SparseFeat('manual_tag_list' + str(x),
                       vocabulary_size=TAG_MAX ,
                       embedding_dim=embedding_dim) for x in range(4)  # 
        ] + [SparseFeat('manual_keyword_list' + str(x),
                       vocabulary_size=KEY_WORDS_MAX,
                       embedding_dim=embedding_dim) for x in range(4)
        ] + [SparseFeat('machine_keyword_list' + str(x),
                       vocabulary_size=KEY_WORDS_MAX,
                       embedding_dim=embedding_dim) for x in range(4)
        ]
        
        
        ### dense feature
        dense_feature_columns = [DenseFeat(feat, 1) for feat in dense_features]

        dnn_feature_columns = fixlen_feature_columns + tag_columns + key_words_columns + dense_feature_columns + userid_columns
        return dnn_feature_columns

In [6]:
def trainer(train_model,train_loader,val_loader,epochs,model_path,load_model = False):
    if load_model:
        train_model.load_weights(model_path)
        print('load weights from %s success!' ,model_path)
    epochs = 1 if DEBUG else epochs
    best_score = -1
    early_stop = 1
    no_imporove = 0
    print('run...')
    for epoch in range(epochs):
        history = train_model.fit(train_loader,
                                  epochs=1, verbose=1,workers = 4,use_multiprocessing=True,max_queue_size=20)
        pred_ans = train_model.predict_generator(val_loader)
        pred_ans = np.concatenate(pred_ans,1)
        pred_ans = pd.DataFrame(pred_ans,columns=ACTION_LIST)
        weightauc,uaucs = evaluate_deepctr(val_loader.data[ACTION_LIST],pred_ans,val_loader.data['userid'].values,ACTION_LIST)
        if best_score < weightauc:
            best_score = weightauc
            train_model.save_weights(model_path)
            no_imporove = 0    
        else :
            no_imporove += 1
        if no_imporove >= early_stop:
            print('-----stoped on epoch %s ------- ' % (epoch))
            break
    del train_model
    gc.collect()

## get model

In [7]:
def get_Shared_Bottom_base(dnn_feature_columns):
    num_tasks = len(ACTION_LIST)
    train_model = Shared_Bottom(
                       dnn_feature_columns=dnn_feature_columns,
                       num_tasks=num_tasks,
                       bottom_dnn_units=[128,128],
                       task_types = ['binary' for i in range(num_tasks)],
                       task_names = ACTION_LIST,
                       tower_dnn_units_lists = [[64,32] for i in range(num_tasks) ],
    )

    train_model.compile('adagrad', loss='binary_crossentropy')
    return train_model

def get_MMOE_FEFM_mutihead_base(dnn_feature_columns):
    num_tasks = len(ACTION_LIST)
    train_model = MMOE_FefM_multihead(
                   dnn_feature_columns=dnn_feature_columns,
                   num_tasks=num_tasks,
                   task_types = ['binary' for i in range(num_tasks)],
                   task_names = ACTION_LIST,
                   num_experts=7,
                   tower_dnn_units_lists = [[64,32] for i in range(num_tasks) ],
                   dnn_hidden_units=(128, 128),
                    multi_head_num = 5,
                   expert_dim=32,)
    train_model.compile('adagrad', loss='binary_crossentropy')
    return train_model

def get_PLE_CGC_FEFM_base(dnn_feature_columns):
    num_tasks = len(ACTION_LIST)
    train_model = PLE_CGC_FEFM(dnn_feature_columns = dnn_feature_columns,
                              num_tasks = num_tasks,
                              task_types = ['binary' for i in range(num_tasks)],
                               task_names = ACTION_LIST,
                               num_experts_specific = 2,
                               num_experts_shared = 4,
                               expert_dnn_units = [128,128] ,
                               gate_dnn_units = [64],
                               tower_dnn_units_lists = [[64,32] for i in range(num_tasks) ],
                              )
    train_model.compile('adagrad', loss='binary_crossentropy')
    return train_model


In [8]:
DEBUG = False
data = loadPkl(USER_ACTION)
data = data.head(1000000) if DEBUG else data

train = data[data.date_ != 14]
val = data[data.date_ ==14]

train_loader = myDataGenerator_base(train,batch_size=4096,mode='train')
val_loader = myDataGenerator_base(val,batch_size=4096 * 4,shuffle = False,mode='test') # shuffle 必须为False
data_loader = myDataGenerator_base(data,batch_size=4096,mode = 'train')
dnn_feature_columns = train_loader.dnn_feature_columns

feedinfo loading over...
feature columns have saved
shuffle data index ing...
feedinfo loading over...
load feature columns /home/tione/notebook/src/model/feature_columns_base.pkl
feedinfo loading over...
feature columns have saved
shuffle data index ing...


In [ ]:
model = get_MMOE_FEFM_mutihead_base(dnn_feature_columns)
trainer(train_model=model, 
        train_loader=train_loader, 
        val_loader=val_loader, 
        epochs=5,
        model_path=os.path.join(MODEL_PATH, 'tf_models/MMOE_FEFM_base/model_seed%s' % (SEED)), 
        load_model=False)


trainer(train_model=model, 
        train_loader=data_loader, 
        val_loader=val_loader, 
        epochs=1,
        model_path=os.path.join(MODEL_PATH, 'tf_models/MMOE_FEFM_base/model_seed%s' % (SEED)), 
        load_model=True)
del model
gc.collect()

dnn input shape (?, 511)
run...
12323/18013 [===================>..........] - ETA: 11:43 - loss: 0.2762 - read_comment_loss: 0.0929 - like_loss: 0.0937 - click_avatar_loss: 0.0377 - forward_loss: 0.0213 - comment_loss: 0.0036 - follow_loss: 0.0054 - favorite_loss: 0.0078

In [ ]:
model = get_PLE_CGC_FEFM_base(dnn_feature_columns)
trainer(train_model=model, 
        train_loader=train_loader, 
        val_loader=val_loader, 
        epochs=5,
        model_path=os.path.join(MODEL_PATH, 'tf_models/PLE_CGC_FEFM_base/model_seed%s' % (SEED)), 
        load_model=False)

trainer(train_model=model, 
        train_loader=data_loader, 
        val_loader=val_loader, 
        epochs=1,
        model_path=os.path.join(MODEL_PATH, 'tf_models/PLE_CGC_FEFM_base/model_seed%s' % (SEED)), 
        load_model=True)
del model
gc.collect()

In [ ]:
model = get_Shared_Bottom_base(dnn_feature_columns)
trainer(train_model=model, 
        train_loader=train_loader, 
        val_loader=val_loader, 
        epochs=5,
        model_path=os.path.join(MODEL_PATH, 'tf_models/share_bottom_base/model_seed%s' % (SEED)), 
        load_model=False)

trainer(train_model=model, 
        train_loader=data_loader, 
        val_loader=val_loader, 
        epochs=1,
        model_path=os.path.join(MODEL_PATH, 'tf_models/share_bottom_base/model_seed%s' % (SEED)), 
        load_model=True)
del model
gc.collect()

# online

In [ ]:
def infer(test_loader,model,model_weights_path,):
    t1 = time.time()
    sub = test_loader.data.copy()
    model.load_weights(model_weights_path)
    print('model weights load from %s' % (model_weights_path))
    pred_ans = model.predict(test_loader,workers = 4,use_multiprocessing=True,max_queue_size=200)
    for i, action in enumerate(ACTION_LIST):
        sub[action] = pred_ans[i]
    t2 = time.time()
    print('7个目标行为%d条样本预测耗时（毫秒）：%.3f' % (len(test), (t2 - t1) * 1000.0))
    ts = (t2 - t1) * 1000.0 / len(test) * 2000.0
    print('7个目标行为2000条样本平均预测耗时（毫秒）：%.3f' % ts)
    return sub[['userid', 'feedid'] + ACTION_LIST]

In [ ]:
import time
if __name__ == "__main__":
    argv = sys.argv
    argv = ['python','submit','../../data/wedata/wechat_algo_data2/test_a.csv']
#     params = xdeepfm_params
    t = time.time() 
    stage = argv[1]
    print('Stage: %s'%stage)
    test_path = ''
    if len(argv)==3:
        test_path = argv[2]
        t1 = time.time()
        test = pd.read_csv(test_path)
        test[ACTION_LIST] = 0
        test_loader = myDataGenerator(test,shuffle=False,batch_size=4096*40,mode ='test')
        dnn_feature_columns = test_loader.dnn_feature_columns
        print('Get test input cost: %.4f s'%(time.time()-t1))
    
    eval_dict = {}
    predict_dict = {}
    predict_time_cost = {}
    ids = None
    
    print('开始预测share bottom...')
    share_bottom_model = get_Shared_Bottom(dnn_feature_columns)
    submission1 = infer(test_loader,share_bottom_model,os.path.join(MODEL_PATH,'tf_models/share_bottom/model_seed%s' % (SEED)))
    
    print('开始预测MMOE FEFM...')
    mmoe_fefm_model = get_MMOE_FEFM(dnn_feature_columns)
    submission2 = infer(test_loader,mmoe_fefm_model,os.path.join(MODEL_PATH,'tf_models/MMOE_FEFM/model_seed%s' % (SEED)))
    
    print('开始预测MMOE MUTI_HEAD...')
    mmoe_multihead_model = get_MMOE_MutiHead(dnn_feature_columns)
    submission3 = infer(test_loader,mmoe_multihead_model,os.path.join(MODEL_PATH,'tf_models/MMOE_MutiHead/model_seed%s' % (SEED)))
    
#     print('开始预测MMOE FEFM...')
#     mmoe_fefm_model = get_MMOE_FEFM(dnn_feature_columns)
#     submission2 = infer(test_loader,mmoe_fefm_model,os.path.join(MODEL_PATH,'tf_models/MMOE_FEFM/model_seed%s' % (SEED)))
    
    
    print('Time cost: %.2f s'%(time.time()-t))

In [ ]:
submission1.to_csv(os.path.join(SUMIT_DIR,'share_bottom.csv'),index=None)
submission2.to_csv(os.path.join(SUMIT_DIR,'MMOE_FEFM.csv'),index=None)
submission3.to_csv(os.path.join(SUMIT_DIR,'MMOE_MutiHead.csv'),index=None)